In [21]:
import sys

In [22]:
sys.path.insert(0,'src')

In [23]:
from repl import *

In [43]:
#深圳
city='shenzhen'
url='http://apply.sztb.gov.cn/apply/app/status/norm/person'

#杭州
city='hangzhou'
url='http://apply.hzcb.gov.cn/apply/app/status/norm/person'

#广州
#city='guangzhou'
#url='http://apply.gzjt.gov.cn/apply/norm/personQuery.html'

In [44]:
post_format='pageNo={1}&issueNumber={0}&applyCode='
post_example='pageNo=8&issueNumber=201607&applyCode='

In [45]:
issue= spider('issue')

In [46]:
_issues=issue.visit(url,post_data=post_example).py_query()('#issueNumber')
issues= [r.text for r in _issues.children()]

In [47]:
s=spider('list')

In [50]:
s.visit(url, post_data=post_example) \
        .search_xpath('\d{13}', 'id',mode='re') \
        .search_xpath('^李[\u4e00-\u9fa5]{2}$', 'name',mode='re').accept().test().get()

id  : /html/body/div[2]/div/div[2]/div/div/table[2]/tbody/tr[2]/td[1]
name  : /html/body/div[2]/div/div[2]/div/div/table[2]/tbody/tr[6]/td[2]


,id,name
0,0929101590072,毛俊
1,0941101381223,林庆福
2,0944101182174,沈淑雯
3,0955101588457,鲁方正
4,0967100614980,李船青
5,0984100158458,俞凯青
6,0986100859557,李梦婕
7,1006100234164,姚文威
8,1007101012162,王涤非
9,1014101228154,朱丽华


In [51]:
c=get_default_connector()

In [52]:
t=task('main')

In [53]:
t.clear()
t.pyge('issue', script=issues)
t.merge('issue:post', script=post_format, merge_with='2')
t.addnew('url', value=url)
t.pl()
t.crawler('url', selector='issue', post_data='[post]')
t.xpath('Content:page',script='/html/head/script[2]',mode='html')
t.html('page',mode='decode')
t.regex('page',script="'\d+'",index=2)
t.number('page')
t.delete('Content post')
t.rangege('p',max='[page]',mode='cross')
t.merge('issue:post',script=post_format,merge_with='p')
t.crawler('url',selector='list',post_data='[post]',new_col='issue')
t.dbex('id',connector='mongo',table='%s_yaohao'%(city))

In [55]:
t.get()

,id,issue,name
0,0000100468883,201608,王丽萍
1,0002101560364,201608,胡晓雯
2,0003100656649,201608,章艺
3,0006100180164,201608,徐亮
4,0007100256162,201608,胡铖璐
5,0011100016017,201608,傅国荣
6,0016101658264,201608,张定伟
7,0036100328447,201608,郑恬
8,0051101014691,201608,江腾
9,0057101280585,201608,裘旦土


In [35]:
#t.check()

1, MergeTF issue:post, issue
2, AddNewTF url, post,issue
3, ParallelTF , post,issue,url
4, CrawlerTF url, post,issue,url
5, XPathTF Content:page, issue,post,url,Content
6, HtmlTF page, issue,page,post,url,Content
7, RegexTF page, issue,page,post,url,Content
8, NumberTF page, issue,page,post,url,Content
9, DeleteTF Content post, issue,page,post,url,Content
10, RangeGE p, issue,page,url
11, MergeTF issue:post, 
12, CrawlerTF url, 
13, DbEX id, 


In [ ]:
t.distribute()

In [ ]:
t.stop_server()